In [2]:
!pip install rasterio
import numpy as np
import rasterio
import pandas as pd
import math
import os


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def dn_to_radiance(dn, Lmax, Lmin=0.0, Qmax=255.0):
    return ((Lmax - Lmin) / Qmax) * dn + Lmin

In [4]:
def radiance_to_reflectance(L, d, Esun, sun_elevation_deg):
    theta_s_rad = math.radians(90 - sun_elevation_deg)
    return (np.pi * L * d**2) / (Esun * np.cos(theta_s_rad))

In [5]:
def extract_band_meta(meta_path):
    lmax = {}
    lmin = {}
    sun_elevation = None

    with open(meta_path, 'r') as f:
        for line in f:
            if 'SunElevationAtCenter' in line:
                sun_elevation = float(line.split('=')[-1].strip())
            if 'B2_Lmax' in line:
                lmax['B2'] = float(line.split('=')[-1].strip())
            if 'B3_Lmax' in line:
                lmax['B3'] = float(line.split('=')[-1].strip())
            if 'B4_Lmax' in line:
                lmax['B4'] = float(line.split('=')[-1].strip())
            if 'B2_Lmin' in line:
                lmin['B2'] = float(line.split('=')[-1].strip())
            if 'B3_Lmin' in line:
                lmin['B3'] = float(line.split('=')[-1].strip())
            if 'B4_Lmin' in line:
                lmin['B4'] = float(line.split('=')[-1].strip())

    return sun_elevation, lmax, lmin

In [6]:
def get_earth_sun_distance(excel_path, doy):
    df = pd.read_excel(excel_path)
    df.columns = df.columns.str.strip()
    return float(df[df['DOY'] == doy]['ESD (AU)'].values[0])

In [ ]:
def convert_image_to_toa_reflectance(
    image_path, band_meta_path, earth_sun_path, output_path, doy
):
    # Load image
    with rasterio.open(image_path) as src:
        bands = src.read().astype(np.float32)  # shape: (3, H, W)
        meta = src.meta.copy()

    # Load metadata
    sun_elevation, lmax_dict, lmin_dict = extract_band_meta(band_meta_path)

    # Load Earth–Sun distance
    d = get_earth_sun_distance(earth_sun_path, doy)

    # Set Esun for RS-2A L4MX
    esun_dict = {'B2': 185.347, 'B3': 158.262, 'B4': 110.81}

    # Process each band
    toa_bands = []
    for i, band_name in enumerate(['B2', 'B3', 'B4']):
        dn = bands[i]
        L = dn_to_radiance(dn, Lmax=lmax_dict[band_name], Lmin=lmin_dict[band_name])
        refl = radiance_to_reflectance(L, d, esun_dict[band_name], sun_elevation)
        toa_bands.append(refl)

    toa = np.stack(toa_bands).astype(np.float32)

    # Save output
    meta.update(dtype='float32', count=3)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(toa)

    print("✅ Saved TOA Reflectance to:", output_path)

In [ ]:
image_path = "data/raw/R2F01JAN2025071113010900054SSANSTUC00GTDD.tif"
band_meta_path = "data/raw/BAND_META.txt"
earth_sun_path = "data/raw/Earth_Sun_distance.xlsx"
output_path = "data/processed/images/R2F01JAN2025_reflectance.tif"
doy = 1  # Jan 1st

convert_image_to_toa_reflectance(
    image_path, band_meta_path, earth_sun_path, output_path, doy
)  # ✅ DONE
